In [ ]:
### still to add sex and age

In [1]:
import numpy as np
import pandas as pd
import os
import shutil
import sklearn.decomposition
# import qtl_config_utils
import sys

In [2]:
outdir = "/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/eqtl/timepoint_D52/input_files_untreated/celltype_DA/extra_runs_addressing_reviews3/"

In [3]:
# general set up
genotypes_file = '/hps/nobackup/hipsci/scratch/genotypes/imputed/REL-2018-01/Full_Filtered_Plink-f/hipsci.wec.gtarray.HumanCoreExome.imputed_phased.20170327.genotypes.norm.renamed.recode.filtered'
# kinship_file = '/hps/nobackup/hipsci/scratch/genotypes/imputed/REL-2018-01/Full_Filtered_Plink-f/hipsci.wec.gtarray.HumanCoreExome.imputed_phased.20170327.genotypes.norm.renamed.recode.filtered.rel'
annotation_file = '/hps/nobackup/hipsci/scratch/singlecell_endodiff/data_processed/scQTLs/annos/ensembl_gene_id_annos.tsv'
#annotation_file = '/hps/nobackup/hipsci/scratch/singlecell_endodiff/data_processed/scQTLs/annos/hgnc_symbol_annos.tsv'
chunk_file = '/nfs/leia/research/stegle/mjbonder/ChunkFiles/Ensembl_75_Limix_Annotation_FC_Gene_step100.txt'
n_pcs = 15
base_dir = '/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/eqtl/timepoint_D52/'

In [4]:
sample_mapping_file = os.path.join(outdir, 'samplemapping.tsv')
phenotype_file = os.path.join(outdir, 'phenotypes.tsv')
covariates_file = os.path.join(outdir, 'covariates.tsv')
noise_matrix_file = os.path.join(outdir, 'noise_matrix.tsv')
# config_file = os.path.join(outdir, 'qtl_config.yaml')

In [5]:
# eQTL discovery parameters
#number of top expressed genes to test
n_genes = 50000

In [6]:
if not os.path.exists(outdir):
    os.makedirs(outdir, exist_ok=True)

In [7]:
mydir = '/nfs/leia/research/stegle/dseaton/hipsci/singlecell_neuroseq/data/data_processed/'
## DA
D52 = 'pool1_17_D52/pool1_17_D52.scanpy.w_metadata.w_celltype.scanpy.celltype.DA.groupedby.donor_id-pool_id-celltype-treatment.mean.tsv'

In [8]:
phenotype_input_file = mydir + D52

In [9]:
# phenotype_input_file = sys.argv[1]
# outdir = sys.argv[2]
# if len(sys.argv)>=4:
#     subset_field = sys.argv[3]
#     selected_subset = sys.argv[4]
# else:
#     subset_field = None
#     selected_subset = None

In [10]:
#load map HGNC to ensembl
mapping_df = pd.read_csv('/nfs/leia/research/stegle/dseaton/genomes/hg19/annotation/geneid_mappings/hgnc_symbol2ensembl_gene_id.txt', sep='\t')

In [11]:
# process phenotype dataframe (untreated)
groupedby_df = pd.read_csv(phenotype_input_file, sep='\t')
groupedby_df = groupedby_df.query('treatment=="NONE"')

In [12]:
groupedby_df.head()

,donor_id,pool_id,celltype,treatment,n_cells,AP006222.2,RP11-206L10.3,RP11-206L10.9,LINC00115,FAM41C,...,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-CYB,AC011043.1,AL592183.1,SRSF10-1
0,HPSI0114i-bezi_3,pool10,DA,NONE,66,0.373148,0.010521,0.034687,0.012997,0.014373,...,3.452707,1.843407,0.599258,3.306375,2.241297,0.292286,3.334886,0.049472,0.382724,0.076166
2,HPSI0114i-eipl_1,pool1,DA,NONE,189,0.362936,0.006138,0.015826,0.026636,0.023932,...,3.618761,2.037559,0.564256,3.442609,2.081173,0.198851,3.564958,0.000000,0.000000,0.093761
4,HPSI0114i-fikt_3,pool14,DA,NONE,26,0.269007,0.000000,0.062557,0.000000,0.000000,...,3.209428,2.040483,0.669133,3.445064,2.108070,0.153443,3.353447,0.032703,0.161917,0.102277
6,HPSI0114i-kolf_2,pool10,DA,NONE,40,0.391692,0.000000,0.018476,0.065576,0.067277,...,3.440805,1.465916,0.297830,3.218524,2.225487,0.130202,2.929145,0.081922,0.045592,0.037953
8,HPSI0114i-kolf_2,pool15,DA,NONE,507,0.330450,0.008953,0.019507,0.014054,0.018184,...,3.714630,1.703519,0.547933,3.542038,2.397292,0.151074,3.211312,0.053037,0.033045,0.002556


In [13]:
phenotype_df = groupedby_df.copy()
phenotype_df.shape[0]

202

In [14]:
phenotype_df.drop_duplicates(subset = "donor_id", inplace = True)
phenotype_df.shape[0]

175

In [15]:
phenotype_df.head()

,donor_id,pool_id,celltype,treatment,n_cells,AP006222.2,RP11-206L10.3,RP11-206L10.9,LINC00115,FAM41C,...,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-CYB,AC011043.1,AL592183.1,SRSF10-1
0,HPSI0114i-bezi_3,pool10,DA,NONE,66,0.373148,0.010521,0.034687,0.012997,0.014373,...,3.452707,1.843407,0.599258,3.306375,2.241297,0.292286,3.334886,0.049472,0.382724,0.076166
2,HPSI0114i-eipl_1,pool1,DA,NONE,189,0.362936,0.006138,0.015826,0.026636,0.023932,...,3.618761,2.037559,0.564256,3.442609,2.081173,0.198851,3.564958,0.000000,0.000000,0.093761
4,HPSI0114i-fikt_3,pool14,DA,NONE,26,0.269007,0.000000,0.062557,0.000000,0.000000,...,3.209428,2.040483,0.669133,3.445064,2.108070,0.153443,3.353447,0.032703,0.161917,0.102277
6,HPSI0114i-kolf_2,pool10,DA,NONE,40,0.391692,0.000000,0.018476,0.065576,0.067277,...,3.440805,1.465916,0.297830,3.218524,2.225487,0.130202,2.929145,0.081922,0.045592,0.037953
10,HPSI0114i-lexy_2,pool10,DA,NONE,50,0.292542,0.018228,0.014019,0.015915,0.000000,...,3.354824,2.325662,0.638373,3.258103,1.927300,0.300389,3.337487,0.000000,0.038658,0.055498


In [16]:
# should only be cells from one celltype
assert(len(phenotype_df['celltype'].drop_duplicates())==1)

In [17]:
#create merged index
phenotype_df['index'] = phenotype_df['donor_id']#+"."+phenotype_df['pool_id']
cell_count_ds = phenotype_df['n_cells']
cell_count_ds.index = phenotype_df['index'].tolist()

In [18]:
#take donor and pool cols for samplemapping df
samplemapping_df = phenotype_df[['donor_id']]
samplemapping_df.head()
samplemapping_df.to_csv(sample_mapping_file, sep='\t', index=False, header=False)

In [19]:
#reorganise to just be expression data indexed by the merged index
cols_to_drop = list(set(phenotype_df.columns) & {'donor_id','celltype','n_cells','treatment','pool_id'})
phenotype_df = phenotype_df.drop(cols_to_drop, axis=1).set_index('index')
phenotype_df = phenotype_df.transpose()
phenotype_df.head()

index,HPSI0114i-bezi_3,HPSI0114i-eipl_1,HPSI0114i-fikt_3,HPSI0114i-kolf_2,HPSI0114i-lexy_2,HPSI0114i-posc_1,HPSI0114i-rozh_4,HPSI0114i-vass_1,HPSI0114i-wegi_1,HPSI0114i-zapk_2,...,HPSI1113i-hayt_3,HPSI1113i-ieki_3,HPSI1113i-podx_1,HPSI1113i-qolg_1,HPSI1113i-qorq_1,HPSI1113i-vuud_2,HPSI1114i-kuul_1,HPSI1114i-ualf_6,HPSI1114i-zuuy_4,HPSI1213i-hehd_1
AP006222.2,0.373148,0.362936,0.269007,0.391692,0.292542,0.376424,0.343145,0.324519,0.171219,0.294685,...,0.301676,0.223509,0.405788,0.183824,0.276859,0.375452,0.317608,0.364334,0.605280,0.306149
RP11-206L10.3,0.010521,0.006138,0.000000,0.000000,0.018228,0.000000,0.000000,0.000000,0.011423,0.002243,...,0.008361,0.000000,0.004511,0.000000,0.008668,0.013400,0.000000,0.000000,0.000000,0.007563
RP11-206L10.9,0.034687,0.015826,0.062557,0.018476,0.014019,0.014933,0.015792,0.000000,0.005841,0.009242,...,0.009852,0.000000,0.008436,0.000000,0.026287,0.013560,0.000000,0.000000,0.025472,0.004336
LINC00115,0.012997,0.026636,0.000000,0.065576,0.015915,0.055751,0.037459,0.022826,0.000000,0.006937,...,0.007126,0.000000,0.010891,0.000000,0.013481,0.011352,0.022266,0.000000,0.122520,0.007679
FAM41C,0.014373,0.023932,0.000000,0.067277,0.000000,0.017594,0.000000,0.029933,0.021134,0.017546,...,0.010255,0.000000,0.010064,0.000000,0.008824,0.012704,0.009125,0.056234,0.063463,0.005390


In [20]:
gene_names = phenotype_df.index

In [21]:
#number of top expressed genes
selected_genes = list(phenotype_df.mean(axis=1).nlargest(n_genes).index)
phenotype_df = phenotype_df.loc[selected_genes, :]

In [22]:
phenotype_list = phenotype_df.index
mapping_df = mapping_df.query('hgnc_symbol in @phenotype_list')
mapping_df = mapping_df.drop_duplicates(subset=['hgnc_symbol'])
mapping_df = mapping_df.set_index('hgnc_symbol')

In [23]:
# limit only to hgnc symbols that map to ensembl gene IDs
phenotype_df = phenotype_df.loc[mapping_df.index,:]
phenotype_df.index = mapping_df['ensembl_gene_id']
phenotype_df.head()

index,HPSI0114i-bezi_3,HPSI0114i-eipl_1,HPSI0114i-fikt_3,HPSI0114i-kolf_2,HPSI0114i-lexy_2,HPSI0114i-posc_1,HPSI0114i-rozh_4,HPSI0114i-vass_1,HPSI0114i-wegi_1,HPSI0114i-zapk_2,...,HPSI1113i-hayt_3,HPSI1113i-ieki_3,HPSI1113i-podx_1,HPSI1113i-qolg_1,HPSI1113i-qorq_1,HPSI1113i-vuud_2,HPSI1114i-kuul_1,HPSI1114i-ualf_6,HPSI1114i-zuuy_4,HPSI1213i-hehd_1
ensembl_gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000261657,0.199801,0.170441,0.135957,0.119144,0.189537,0.074559,0.247768,0.178011,0.181703,0.172295,...,0.152243,0.200721,0.132905,0.229570,0.180572,0.158088,0.149935,0.245467,0.090187,0.205873
ENSG00000227640,0.000000,0.020137,0.000000,0.017075,0.013281,0.000000,0.053130,0.050767,0.000000,0.013735,...,0.021325,0.000000,0.012362,0.061906,0.096830,0.018867,0.000000,0.000000,0.000000,0.041669
ENSG00000247400,0.131556,0.121530,0.340275,0.134915,0.119536,0.062595,0.091913,0.117875,0.095284,0.108437,...,0.124991,0.062933,0.145154,0.089358,0.074252,0.166735,0.089448,0.245197,0.210142,0.166437
ENSG00000228889,0.019465,0.047815,0.068368,0.016302,0.031981,0.026038,0.011520,0.035781,0.019056,0.024619,...,0.029089,0.000000,0.035906,0.176883,0.023026,0.023460,0.035566,0.041154,0.034466,0.045126
ENSG00000261125,0.069042,0.052439,0.033319,0.030723,0.000000,0.083186,0.066110,0.070324,0.038508,0.036354,...,0.063712,0.000000,0.078829,0.000000,0.028379,0.059444,0.047389,0.106279,0.025472,0.049448


In [24]:
phenotype_df.to_csv(phenotype_file, sep='\t')

In [25]:
pc_mat = sklearn.decomposition.PCA(n_components=n_pcs).fit_transform(phenotype_df.values.transpose())
pc_df = pd.DataFrame(data=pc_mat, index=phenotype_df.columns, columns=['PC{}'.format(x) for x in range(1,n_pcs+1)])
pc_df.head(2)
# pc_df.to_csv(covariates_file, sep='\t')

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15
index,,,,,,,,,,,,,,,
HPSI0114i-bezi_3,1.031743,-4.391966,-1.317521,0.886155,-2.318068,2.801095,-2.084242,2.035992,-0.734834,-1.346092,-2.340430,0.897605,0.397409,0.279368,0.075659
HPSI0114i-eipl_1,-1.567021,1.141687,-0.011901,2.940767,0.245461,-2.369905,-0.293510,1.623764,1.118979,-0.558705,-1.171336,-0.901630,-0.469328,0.057526,0.457139


In [26]:
batch_df = groupedby_df.copy()
batch_df.drop_duplicates(subset = "donor_id", inplace = True)
batch_df = batch_df.drop(gene_names, axis=1)
batch_df['index'] = batch_df['donor_id']#+"."+cov_df['pool_id']
cols_to_drop = list({'donor_id','celltype','treatment','n_cells'})
batch_df = batch_df.drop(cols_to_drop, axis=1).set_index('index')
batch_df.head()

,pool_id
index,
HPSI0114i-bezi_3,pool10
HPSI0114i-eipl_1,pool1
HPSI0114i-fikt_3,pool14
HPSI0114i-kolf_2,pool10
HPSI0114i-lexy_2,pool10


In [27]:
num_rows = batch_df.shape[0]
pool_list = batch_df['pool_id'].unique()
pool_membership = [ np.reshape((batch_df['pool_id'].values == elem).astype(np.float64), (num_rows,1)) for elem in pool_list ]
design_matrix_pool = np.hstack(tuple(pool_membership))
design_df = pd.DataFrame(design_matrix_pool, index=batch_df.index, columns=pool_list)
design_df.head()

,pool10,pool1,pool14,pool17,pool6,pool15,pool4,pool8,pool2,pool5,pool7,pool9,pool16,pool3,pool13,pool11
index,,,,,,,,,,,,,,,,
HPSI0114i-bezi_3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HPSI0114i-eipl_1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HPSI0114i-fikt_3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HPSI0114i-kolf_2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HPSI0114i-lexy_2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
cov_df = design_df.copy()
for i in range(15):
    cov_df[['PC'+str(i+1)]] = pc_df[['PC'+str(i+1)]] 
cov_df.head()
# cov_df.to_csv(covariates_file, sep='\t')

,pool10,pool1,pool14,pool17,pool6,pool15,pool4,pool8,pool2,pool5,...,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15
index,,,,,,,,,,,,,,,,,,,,,
HPSI0114i-bezi_3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.801095,-2.084242,2.035992,-0.734834,-1.346092,-2.340430,0.897605,0.397409,0.279368,0.075659
HPSI0114i-eipl_1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.369905,-0.293510,1.623764,1.118979,-0.558705,-1.171336,-0.901630,-0.469328,0.057526,0.457139
HPSI0114i-fikt_3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.039783,8.379162,5.410001,-3.318785,-1.895812,0.365705,0.811733,-2.508071,0.929687,1.028481
HPSI0114i-kolf_2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.661853,-0.706521,-0.510278,-2.031429,-1.454100,-0.902277,-0.197888,0.324657,-1.591523,0.118494
HPSI0114i-lexy_2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.845287,-1.637854,0.803109,-1.036410,-0.750399,-2.384434,0.663404,-0.726501,0.701786,-0.047737


In [43]:
metadata_df = pd.read_csv('/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/metadata_all_lines.csv', index_col = 0)
metadata_df.index = metadata_df['donor_id']
metadata_df.head(2)

,donor_id,Gender,Age,feederfree,min_age,max_age,avg_age
donor_id,,,,,,,
HPSI0813i-ffdc_1,HPSI0813i-ffdc_1,male,40-44,n,40.0,44.0,42.0
HPSI0213i-nawk_55,HPSI0213i-nawk_55,male,NaN,y,NaN,NaN,NaN


In [44]:
metadata_df.shape[0]

215

In [45]:
num_rows = metadata_df.shape[0]
gender_list = metadata_df['Gender'].unique()
gender_membership = [ np.reshape((metadata_df['Gender'].values == elem).astype(np.float64), (num_rows,1)) for elem in gender_list ]
meta_matrix_pool = np.hstack(tuple(gender_membership))
meta_df = pd.DataFrame(meta_matrix_pool, index=metadata_df['donor_id'], columns=gender_list)
meta_df.head()
meta_df.shape[0]

215

In [47]:
meta_df['avg_age'] = metadata_df['avg_age']
meta_df.head()

,male,female,avg_age
donor_id,,,
HPSI0813i-ffdc_1,1.0,0.0,42.0
HPSI0213i-nawk_55,1.0,0.0,NaN
HPSI0513i-uoxz_4,0.0,1.0,NaN
HPSI0513i-veve_2,0.0,1.0,NaN
HPSI0413i-peop_4,1.0,0.0,NaN


In [48]:
meta_df = meta_df.sort_values('donor_id')
meta_df.index.names = ['index'] 
meta_df.head()

,male,female,avg_age
index,,,
HPSI0114i-bezi_3,0.0,1.0,62.0
HPSI0114i-eipl_1,0.0,1.0,42.0
HPSI0114i-fikt_3,1.0,0.0,52.0
HPSI0114i-kolf_2,1.0,0.0,57.0
HPSI0114i-lexy_2,0.0,1.0,62.0


In [56]:
cov_df = pd.merge(meta_df, design_df, left_index=True, right_index=True)
cov_df = cov_df.dropna(axis=0)
cov_df.head()
# cov_df.shape[0]

,male,female,avg_age,pool10,pool1,pool14,pool17,pool6,pool15,pool4,pool8,pool2,pool5,pool7,pool9,pool16,pool3,pool13,pool11
index,,,,,,,,,,,,,,,,,,,
HPSI0114i-bezi_3,0.0,1.0,62.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HPSI0114i-eipl_1,0.0,1.0,42.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HPSI0114i-fikt_3,1.0,0.0,52.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HPSI0114i-kolf_2,1.0,0.0,57.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HPSI0114i-lexy_2,0.0,1.0,62.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
cov_df.to_csv(covariates_file, sep='\t')

In [58]:
noise_scaling_vector = [1/float(x) for x in cell_count_ds.tolist()]
noise_matrix = np.diag(noise_scaling_vector)
noise_matrix_df = pd.DataFrame(data=noise_matrix, index=cell_count_ds.index, columns=cell_count_ds.index)
noise_matrix_df.head()

,HPSI0114i-bezi_3,HPSI0114i-eipl_1,HPSI0114i-fikt_3,HPSI0114i-kolf_2,HPSI0114i-lexy_2,HPSI0114i-posc_1,HPSI0114i-rozh_4,HPSI0114i-vass_1,HPSI0114i-wegi_1,HPSI0114i-zapk_2,...,HPSI1113i-hayt_3,HPSI1113i-ieki_3,HPSI1113i-podx_1,HPSI1113i-qolg_1,HPSI1113i-qorq_1,HPSI1113i-vuud_2,HPSI1114i-kuul_1,HPSI1114i-ualf_6,HPSI1114i-zuuy_4,HPSI1213i-hehd_1
HPSI0114i-bezi_3,0.015152,0.000000,0.000000,0.000,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HPSI0114i-eipl_1,0.000000,0.005291,0.000000,0.000,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HPSI0114i-fikt_3,0.000000,0.000000,0.038462,0.000,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HPSI0114i-kolf_2,0.000000,0.000000,0.000000,0.025,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HPSI0114i-lexy_2,0.000000,0.000000,0.000000,0.000,0.02,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
noise_matrix_df.to_csv(noise_matrix_file, sep='\t')
# kinship_file = noise_matrix_file

In [104]:
# number_of_permutations = '1000'
# minor_allele_frequency = '0.05'
# hwe = '0.000001'
# call_rate = '1'
# window_size = '250000'
# block_size = '15000'


# config_dict = {'af': annotation_file,
#                'pf': phenotype_file,
#                'cf': covariates_file,
#                'kf': kinship_file,
#                'smf': sample_mapping_file,
#                'plink': genotypes_file,
#                'maf': minor_allele_frequency,
#                'hwe': hwe,
#                'np': number_of_permutations,
#                'cr': call_rate,
#                'w': window_size,
#                'bs': block_size,
#                'chunk_file': chunk_file
# }

# # write config to a file
# qtl_config_utils.write_config(config_dict, config_file)